In [26]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [27]:
env = GridworldEnv()

In [28]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    
    '''
        My comments:
        
        iterative policy evaluation iterativley applies the bellman expecation eqn 
        V(s) = sum_a p(a|s) * sum_{s',r}p(s',r|s,a)[r + gamma * V(s')]
    '''
    
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        delta = 0
        for state in range(env.nS):
            v = V[state]
            sum = 0
            for action,prob_action in enumerate(policy[state]):
                for prob,next_state,reward,_ in env.P[state][action]:
                    sum += prob_action*(prob*(reward + discount_factor*V[next_state]))
            V[state] = sum
            delta = max(delta,abs(v-V[state]))
        if delta < theta:
            break
    return np.array(V)

In [29]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [30]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)